# CoT & ReACT Prompting Introcution
React Prompting verbindet die Vorteile von LLMs "Reasoning" und erweitert sie mit weiteren Funktionalitäten, die normale LLMs nicht können. ACT

In [2]:
import os
import json
import requests
from openai import AzureOpenAI
from dotenv import load_dotenv

# --- 1. Konfiguration und Initialisierung ---

# Lade alle Umgebungsvariablen aus der .env Datei
load_dotenv()

# Konfiguration für Azure OpenAI
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

if not api_key or not azure_endpoint or not azure_deployment:
    raise ValueError("Azure OpenAI Konfiguration fehlt. Bitte überprüfe deine .env Datei.")

# Initialisiere den Azure OpenAI Client
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint,
)

# Das Problem
LLMs sind reine Sprachmodelle sind stateless und haben keine Zugang zu externen Tools. ChatGPT ist ein Finegetunetes LLM mit Tool Access und geht weit über ein herkömmliches LLM hinaus, viele sprechen bei ChatGPT bereits von einer Application.

In [3]:
basic_user_prompt = "Wie ist das Wetter gerade in Dättwil?"
basic_system_prompt = "Du bist ein hilfreicher Assistent."


messages = [
        {"role": "system", "content": basic_system_prompt},
        {"role": "user", "content": basic_user_prompt}
    ]

response = client.chat.completions.create(
    model=azure_deployment,
    messages=messages,
    temperature=0
)   

print(response.choices[0].message.content)

Es tut mir leid, aber ich kann keine aktuellen Wetterdaten abrufen. Für die aktuelle Wetterlage in Dättwil empfehle ich, eine Wetter-Website oder eine Wetter-App zu nutzen, wie zum Beispiel Wetter.com, MeteoSchweiz oder eine ähnliche Plattform.


# Chain of Thought Prompting
Die Idee hier ist super einfach, aber genial. Anstatt dem LLM nur zu sagen: 'Hier ist das Problem, gib mir die Antwort', zwingen wir es, seinen Denkprozess offenzulegen. Wir sagen quasi: 'Denk laut nach und zeig mir deine Arbeit, Schritt für Schritt

In [4]:
basic_user_prompt = "Wie ist das Wetter gerade in Dättwil?"
basic_system_prompt = """Du bist ein hilfreicher Assistent, der Schritt für Schritt denkt, bevor er antwortet. 
Zerlege das Problem in kleinere Teile und gib Lösungsvorschläge für die einzelnen Schritte.
"""


messages = [
        {"role": "system", "content": basic_system_prompt},
        {"role": "user", "content": basic_user_prompt}
    ]

response = client.chat.completions.create(
    model=azure_deployment,
    messages=messages,
    temperature=0
)   

print(response.choices[0].message.content)

Um das aktuelle Wetter in Dättwil zu erfahren, kannst du folgende Schritte unternehmen:

1. **Online-Wetterdienste nutzen**: 
   - Besuche Websites wie Wetter.com, MeteoSchweiz oder Weather.com.
   - Gib „Dättwil“ in die Suchleiste ein, um die aktuellen Wetterdaten zu erhalten.

2. **Wetter-Apps verwenden**:
   - Öffne eine Wetter-App auf deinem Smartphone (z.B. Wetter.com, AccuWeather, MeteoSwiss).
   - Suche nach „Dättwil“ und schaue dir die aktuellen Wetterbedingungen an.

3. **Sprachassistenten fragen**:
   - Nutze Sprachassistenten wie Siri, Google Assistant oder Alexa und frage: „Wie ist das Wetter gerade in Dättwil?“

Da ich keine Echtzeitdaten abrufen kann, empfehle ich dir, eine der oben genannten Methoden zu verwenden, um die aktuellen Wetterinformationen zu erhalten. Wenn du möchtest, kann ich dir auch eine Anleitung geben, wie du eine Wetter-API nutzen kannst, um das Wetter programmatisch abzufragen.


# Lösung ReACT Prompting
Um unserem LLM, die Fähigkeit zu geben auf Wetterdaten zuzugreifen brauchen wir ein ReACT Framework mit API Function Call zu einem Wetter API.

In [5]:
# Get_weather Funktion
def get_weather(city: str) -> str:
    """Ermittelt das Echtzeit-Wetter für eine Stadt."""
    print(f"[Tool Called]: get_weather(city='{city}')")
    API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')
    if not API_KEY:
        return json.dumps({"error": "Google Maps API Key nicht gefunden."})
    geo_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={city}&key={API_KEY}"
    try:
        geo_response = requests.get(geo_url)
        geo_data = geo_response.json()
        if geo_data['status'] == 'OK':
            location = geo_data['results'][0]['geometry']['location']
            lat, lon = location['lat'], location['lng']
            weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
            weather_response = requests.get(weather_url)
            weather_data = weather_response.json()
            return json.dumps(weather_data.get("current_weather", {}))
        else:
            return json.dumps({"error": "Stadt nicht gefunden."})
    except Exception as e:
        return json.dumps({"error": f"API Fehler: {e}"})

# Final_answer Funktion
def final_answer(summary: str) -> str:
    """Gibt die endgültige Antwort aus."""
    print(f"\n>>> Final Answer: {summary}")
    return "Execution finished."

available_tools = {"get_weather": get_weather, "final_answer": final_answer}


# Hilfsfunktion zum Abrufen von LLM-Antworten
def parse_action(ai_response: str):
    """Extrahiert den Werkzeugaufruf aus der LLM-Antwort."""
    try:
        act_line = next(line for line in ai_response.splitlines() if line.startswith('ACT:'))
        action_str = act_line[len('ACT: '):].strip()
        tool_name = action_str[:action_str.find('(')]
        args_str = action_str[action_str.find('(')+1:-1]
        args = {}
        if '=' in args_str:
            key, value = args_str.split('=', 1)
            args[key.strip()] = value.strip().strip('\"\'')
        return tool_name, args
    except (StopIteration, ValueError):
        return None, None

In [ ]:
# System Prompt für ReAct
system_prompt = """
You are a helpful assistant that can access external tools.
You must use a step-by-step reasoning process. For each step, respond with a single THINK/ACT block.

THINK: Reason about the problem and decide the next logical action.
ACT: Call ONE of the available tools. Use the format `tool_name(parameter="value")`.

# AVAILABLE TOOLS
---
## 1. get_weather(city: str)
* Use this to get the weather for a specific city.
* Example Input: `ACT: get_weather(city="Zurich")`

## 2. final_answer(summary: str)
* Use this tool ONLY when you have all the information to answer the user's request.
* Example Input: `ACT: final_answer(summary="The weather in Zurich is 15°C.")`
---
"""

In [29]:
# --- 4. Die Live ReAct-Schleife (Orchestrator) ---

def main():
    """Führt die Live-Interaktion mit dem LLM aus."""
    user_prompt = "Wie ist das Wetter gerade in Dättwil?"
    print(f"User: {user_prompt}\n")

    # Die Konversationshistorie, die wir an das LLM senden.
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    turn_count = 1
    while turn_count <= 5: # Sicherheitslimit, um Endlosschleifen zu vermeiden
        print(f"--- Turn {turn_count} ---")
        
        # 1. Das LLM "denkt" und generiert eine Antwort
        response = client.chat.completions.create(
            model=azure_deployment,
            messages=messages,
            temperature=0
        )
        ai_response_text = response.choices[0].message.content
        print(f"AI Assistant:\n{ai_response_text}\n")
        messages.append({"role": "assistant", "content": ai_response_text})

        # 2. Wir parsen die Aktion aus der Antwort
        tool_name, args = parse_action(ai_response_text)
        if not tool_name or tool_name not in available_tools:
            print("Could not parse a valid tool. Ending.")
            break

        # 3. Wir führen das Werkzeug aus
        tool_function = available_tools[tool_name]
        observation = tool_function(**args)
        observation_message = f"OBSERVE: {observation}"
        print(f"{observation_message}\n")
        
        # 4. Wir fügen das Ergebnis zur Historie hinzu, damit das LLM im nächsten Schritt darauf reagieren kann
        messages.append({"role": "user", "content": observation_message})

        # 5. Wenn die finale Antwort gegeben wurde, beenden wir die Schleife
        if tool_name == "final_answer":
            break
            
        turn_count += 1

if __name__ == "__main__":
    main()

User: Wie ist das Wetter gerade in Dättwil?

--- Turn 1 ---
AI Assistant:
ACT: get_weather(city="Dättwil")

[Tool Called]: get_weather(city='Dättwil')
OBSERVE: {"time": "2025-09-15T09:30", "interval": 900, "temperature": 20.7, "windspeed": 14.7, "winddirection": 245, "is_day": 1, "weathercode": 3}

--- Turn 2 ---
AI Assistant:
ACT: final_answer(summary="Das aktuelle Wetter in Dättwil ist bewölkt mit einer Temperatur von 20,7°C. Der Wind weht mit einer Geschwindigkeit von 14,7 km/h aus Richtung 245 Grad.")


>>> Final Answer: Das aktuelle Wetter in Dättwil ist bewölkt mit einer Temperatur von 20,7°C. Der Wind weht mit einer Geschwindigkeit von 14,7 km/h aus Richtung 245 Grad.
OBSERVE: Execution finished.

